In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Specify sub_step parameters
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    
    outputs = 
    [
        { ENTITY_NAME: "bento_service" }
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "some_model_binary_artifact" }
    ]

)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# Train
from sklearn import svm
from sklearn import datasets
from bentoservice import TestService

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

# Create a iris classifier service with the newly trained model
test_service = TestService()
test_service.pack("model", clf)
test_service.pack("some_model_binary_artifact", b"model_37")

In [ ]:
# Test model API
from sinara.bentoml import start_dev_bentoservice, stop_dev_bentoservice

# Test invoking BentoService instance
test_service.predict([[5.1, 3.5, 1.4, 0.2]])

# Start a dev model server to test out the API endpoint locally
start_dev_bentoservice(test_service)

# Stop the dev model server
stop_dev_bentoservice(test_service)

In [ ]:
# Serve model
from sinara.bentoml import save_bentoservice

outputs = substep.outputs()

# Save the entire prediction service to a BentoML bundle
save_bentoservice(test_service, path=outputs.bento_service, substep=substep)

In [ ]:
# Load model and save model's artifacts to tmp
from sinara.bentoml import load_bentoservice, save_bentoartifact_to_tmp

tmp_outputs = substep.tmp_outputs()

# read trained model    
bento_service = load_bentoservice(outputs.bento_service) 

# save model's artifact for evaluation
save_bentoartifact_to_tmp(bento_service, "some_model_binary_artifact", tmp_outputs.some_model_binary_artifact)